In [1]:
import sys
sys.path.append('../../../')
from pprint import pprint
import time
import pickle
import importlib
import numpy as np

In [2]:
from qiskit.ignis.mitigation.measurement import TensoredMeasFitter
from lib_grover import make_hit_list
from libs_qrem import IgnisFilter

In [3]:
nbit = 20 # change this value to get discretized result closer to analytical results # 3
#setting the number of shots and Grover operators.
max_iter = 10

In [4]:
with open("pkls/raw_hist_list_list.pkl", "rb") as f:
    raw_hist_list_list = pickle.load(f)
with open("pkls/meas_fitters.pkl", "rb") as f:
    meas_fitters = pickle.load(f)

In [5]:
ignis_mitigator_list = []
for i in range(0, max_iter):
    ignis_mitigator_list.append(IgnisFilter(nbit + 1, meas_fitters[i].cal_matrices))
    if i % 10 == 0:
        print("size", i, "finished")
print("length of ignis_mitigator_list: ", len(ignis_mitigator_list))

size 0 finished
length of ignis_mitigator_list:  10


In [6]:
def get_info(mitigator):
    return {# "exact_one_norm_of_reduced_inv_A": mitigator.exact_one_norm_of_reduced_inv_A(),
            "mitigated_hist": mitigator.mitigated_hist(),
            "times": mitigator.times(),
            # "mitigation_stddev": mitigator.mitigation_stddev(norm_type = "exact"),
            }

In [7]:
ignis_info_list_list = []
ignis_hit_list_list = []
for i, raw_hist_list in enumerate(raw_hist_list_list):
    ignis_info_list = []
    ignis_hist_list = []
    t1 = time.perf_counter()
    for raw_hist in raw_hist_list:
        ignis_hist_list.append( ignis_mitigator_list[i].apply(raw_hist) )
        ignis_info_list.append( get_info(ignis_mitigator_list[i]) )
    ignis_hit_list_list.append( make_hit_list(ignis_hist_list) )
    ignis_info_list_list.append( ignis_info_list )
    t2 = time.perf_counter()
    print("iter", i, "finished", t2 - t1, "s")

iter 0 finished 5.431459689 s
iter 1 finished 5.2022705049999995 s
iter 2 finished 5.460440358000001 s
iter 3 finished 5.761974042999999 s
iter 4 finished 5.574863606000001 s
iter 5 finished 5.944979193999998 s
iter 6 finished 5.662670511000002 s
iter 7 finished 6.0855555229999965 s
iter 8 finished 5.694048081999995 s
iter 9 finished 5.595568053000001 s


In [8]:
with open("pkls/ignis_info_list_list.pkl", "wb") as f:
    pickle.dump(ignis_info_list_list, f)
with open("pkls/ignis_hit_list_list.pkl", "wb") as f:
    pickle.dump(ignis_hit_list_list, f)